In [19]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import GOOG
import talib
import pandas as pd

AVAX = pd.read_csv('AVAX_1h.csv')
AVAX['Date'] = pd.to_datetime(AVAX['Date'], format='%m/%d/%Y %H:%M')
# Set the 'Date' column as the index
AVAX.set_index('Date', inplace=True)
AVAX.sort_index(inplace=True)

# print(AVAX.head())

class MyStrategy(Strategy):
    def init(self):
        self.long_position = False
        self.short_position = False

    def next(self):
        rsi = talib.RSI(self.data.Close, timeperiod=14)
        # Compute moving averages
        ma5 = talib.SMA(self.data.Close, timeperiod=5)
        ma8 = talib.SMA(self.data.Close, timeperiod=8)
        # Compute MACD
        macd, signal, _ = talib.MACD(self.data.Close, fastperiod=12, slowperiod=26, signalperiod=9)

        if (rsi[-1] < 30 or crossover(ma5, ma8) or crossover(macd, signal)) and not self.long_position:
            self.buy()
            self.long_position = True
        elif (rsi[-1] > 70 or crossover(signal, macd)) and self.long_position:
            self.position.close()
            # self.sell()
            # self.short_position = True
            self.long_position = False


bt = Backtest(AVAX, MyStrategy,cash=1000)

output = bt.run()
print(output)
# bt.plot(resample = '1H')

Start                     2020-09-22 06:00:00
End                       2023-07-25 23:00:00
Duration                   1036 days 17:00:00
Exposure Time [%]                   70.481223
Equity Final [$]                    1505.7063
Equity Peak [$]                     4400.5163
Return [%]                           50.57063
Buy & Hold Return [%]               173.09418
Return (Ann.) [%]                   15.542556
Volatility (Ann.) [%]              159.871643
Sharpe Ratio                         0.097219
Sortino Ratio                        0.220265
Calmar Ratio                         0.206525
Max. Drawdown [%]                  -75.257533
Avg. Drawdown [%]                   -9.992377
Max. Drawdown Duration      479 days 11:00:00
Avg. Drawdown Duration       21 days 00:00:00
# Trades                                 1011
Win Rate [%]                        44.312562
Best Trade [%]                      23.193443
Worst Trade [%]                    -41.122744
Avg. Trade [%]                    